# ReAct con LangChain (Avanzado)

## Definición
ReAct combina razonamiento + acción: el agente decide, ejecuta herramientas y usa observaciones para el siguiente paso.

## Cuándo usarlo
- Cuando necesitas herramientas reales (auditoría, búsqueda, validación).
- Cuando la trazabilidad de pasos es requisito de negocio.

## Cuándo NO usarlo
- Si no hay tools que agreguen valor.
- Si una llamada directa ya cumple calidad/costo/latencia.

## Flujo conceptual

```mermaid
graph TD
A[State] --> B[Decide Action]
B --> C[ANALIZAR_PERFIL]
C --> D[GENERAR_MENSAJE]
D --> E[AUDITAR_RESPETO]
E --> F[FINAL_ANSWER]
```

In [ ]:
# Setup de notebook
from __future__ import annotations

import importlib.util
import inspect
import json
import sys
from pathlib import Path
from IPython.display import Markdown, display

# Add project root to path
project_root = Path.cwd()
while not (project_root / "pyproject.toml").exists() and project_root.parent != project_root:
    project_root = project_root.parent
sys.path.insert(0, str(project_root))

def load_module(path: Path, module_name: str):
    spec = importlib.util.spec_from_file_location(module_name, path)
    if spec is None or spec.loader is None:
        raise RuntimeError(f"Cannot load module: {path}")
    module = importlib.util.module_from_spec(spec)
    sys.modules[module_name] = module
    spec.loader.exec_module(module)
    return module

ROOT = project_root
print('Repo root:', ROOT)

In [ ]:
script = ROOT / '03_langchain_prompting/ReAct_LangChain/Notebooks/01_react_langchain_avanzado.py'
module = load_module(script, 'react_langchain_nb')
run_fn = module.run_react_langchain
print('Script:', script)
print('Firma:', inspect.signature(run_fn))

## Código fuente principal (visible)

In [ ]:
print(inspect.getsource(run_fn)[:7000])

## Ejemplo 1: caso base

In [ ]:
result_1 = run_fn(verbose=False)
print('Modelo:', result_1['__model'])
print('Context hash:', result_1['__context_hash'])
print('Final answer:')
print(json.dumps(result_1['final_answer'], ensure_ascii=False, indent=2))
print('Trace preview:')
print(json.dumps(result_1['trace_preview'], ensure_ascii=False, indent=2))

In [ ]:
display(Markdown('## Flujo real usado en ejecución'))
display(Markdown('```mermaid\n' + result_1['__flow_mermaid'] + '\n```'))

## Ejemplo 2: coqueteo (asistente latino experto en enamorar)

In [ ]:
profile_coqueteo = {
  "tipo_persona": "asistente latino experto en enamorar con elegancia",
  "gustos": [
    "salsa romantica",
    "cafes bohemios",
    "poesia urbana",
    "paseos nocturnos"
  ],
  "estilo": "carismatico, coqueto, respetuoso, humor picante sutil",
  "contexto": "quiere iniciar una conversacion de coqueteo sin frases prefabricadas"
}
result_2 = run_fn(profile=profile_coqueteo, verbose=False)
print('Context hash coqueteo:', result_2['__context_hash'])
print('Final answer coqueteo:')
print(json.dumps(result_2['final_answer'], ensure_ascii=False, indent=2))

## Errores típicos en producción
- Tools con contratos ambiguos: el agente se vuelve impredecible.
- Sin guardrails de secuencia: deriva de acciones y debugging costoso.
- Sin auditoría de respeto: riesgo reputacional en outputs sensibles.
- Sobrecargar de herramientas irrelevantes: más latencia, menos precisión.

## Autocrítica
- ReAct aporta control operativo, pero cuesta más que CoT simple.
- La calidad sigue dependiendo del contexto; herramientas malas no rescatan un mal diseño.